In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'
#Tensorflow log management: '1':info, '2':warning, '3': error

import time 

import tensorflow as tf
import tensorflow.contrib.layers as layers
from tensorflow.examples.tutorials.mnist import input_data
#Tutorials have mnist data files and queue

In [ ]:
mnist = input_data.read_data_sets("/data/mnist", one_hot=True)
#MNIST images' has 28*28=764 features. Gray Image: Depth=1

In [ ]:
#Hyperparameters
N_CLASSES = 10
LEARNING_RATE = 0.001
BATCH_SIZE = 128
SKIP_STEP = 10
DROPOUT = 0.75
N_EPOCHS = 1

In [ ]:
#Create Placeholder. X and Y
with tf.name_scope('data'):
    X = ###shape = [None, 784]
    Y = ###shape = [None, 10]

dropout = tf.placeholder(tf.float32, name='dropout') #Dropout Rate

#Global_step for writer
global_step = tf.Variable(0, dtype=tf.int32, trainable=False, name='global_step')

In [ ]:
# Construnct Deep Learning Model conv1-pool1-conv2-pool2-fc-softmax
# first, reshape the image to [BATCH_SIZE, 28, 28, 1] to make it work with tf.nn.conv2d
with tf.variable_scope('conv1') as scope:
    images = ###Reshape input with [BATCH_SIZE, HEIGHT, WEDTH, DEPTH]
    kernel = ###Make 32 number of 5*5 filters, initializer=tf.truncated_normal_initializer()
    biases = ###Make bias term of 32 number of filters, initializer=tf.random_normal_initializer()
    conv = ###Make convolutional layer with conv2d, stride=1, padding=SAME
    conv1 = ###Make activation ReLu of conv

    # (output): [BATCH_SIZE x 28 x 28 x 32]
    # conv1 = layers.conv2d(images, 32, 5, 1, activation_fn=tf.nn.relu, padding='SAME')

with tf.variable_scope('pool1') as scope:
    pool1 = ###Make max pooling layer with 2*2 filter, stride=2, padding=SAME

In [ ]:
with tf.variable_scope('conv2') as scope:
    kernel = ###Make 64 number of 5*5 filters, initializer=tf.truncated_normal_initializer()
    biases = ###Make bias term of 32 number of filters
    conv = ###Make convolutional layer with conv2d, stride=1, padding=SAME
    conv2 = ###Make activation ReLu of conv

with tf.variable_scope('pool2') as scope:
    # similar to pool1
    pool2 = ###Make max pooling layer with 2*2 filter, stride=2, padding=SAME

In [ ]:
with tf.variable_scope('fc') as scope:
    pool2 = ###Flatten pool2 for full-connected layer
    input_features = 7 * 7 * 64
    w = tf.get_variable('weights', [input_features, 1024],
                        initializer=tf.truncated_normal_initializer())
    b = tf.get_variable('biases', [1024],
                        initializer=tf.constant_initializer(0.0))


    fc = tf.nn.relu(tf.matmul(pool2, w) + b, name='relu')
    # fc = layers.fully_connected(pool2, 1024, tf.nn.relu)

    fc = tf.nn.dropout(fc, dropout, name='relu_dropout')

In [ ]:
with tf.variable_scope('softmax_linear') as scope:
    w = tf.get_variable('weights', [1024, N_CLASSES],
                        initializer=tf.truncated_normal_initializer())
    b = tf.get_variable('biases', [N_CLASSES],
                        initializer=tf.random_normal_initializer())
    logits = tf.matmul(fc, w) + b
    #In here, not use softmax function because we will use tf.nn.softmax_cross_entropy_with_logits

In [ ]:
with tf.name_scope('loss'):
    entropy = tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=logits)
    loss = tf.reduce_mean(entropy, name='loss')

In [ ]:
with tf.name_scope('summaries'):
    ###Make summary scalar, recording loss. name='loss'
    ###Make summary histogram, recording loss. name='histogram_loss'
    summary_op = tf.summary.merge_all() 

In [ ]:
optimizer = tf.train.AdamOptimizer(LEARNING_RATE)
train = ###minimize loss

In [ ]:
os.mkdir('./checkpoints')
os.mkdir('./checkpoints/convnet_mnist')

In [ ]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

In [ ]:
# to visualize using TensorBoard
writer = ###Make tensorboard writer in './graphs/convnet'

In [ ]:
saver = ###Declare saver module
ckpt = tf.train.get_checkpoint_state(os.path.dirname('./checkpoints/convnet_mnist/checkpoint'))
# if that checkpoint exists, restore from checkpoint
if ckpt and ckpt.model_checkpoint_path:
    saver.restore(sess, ckpt.model_checkpoint_path)

In [ ]:
initial_step = sess.run(global_step)

start_time = time.time()
n_batches = int(mnist.train.num_examples / BATCH_SIZE)

total_loss = 0.0
for index in range(initial_step, n_batches * N_EPOCHS): # train the model n_epochs times
    X_batch, Y_batch = mnist.train.next_batch(BATCH_SIZE)
    _, loss_batch, summary = ###run train, loss, summary_op with Feed_dict
    ### Add summary result to writer, global_variable=index (why?)
    total_loss += loss_batch
    if (index + 1) % SKIP_STEP == 0:
        print('Average loss at step {}: {:5.1f}'.format(index + 1, total_loss / SKIP_STEP))
        total_loss = 0.0
        saver.save(sess, 'checkpoints/convnet_mnist/mnist-convnet', index)

print("Optimization Finished!") # should be around 0.35 after 25 epochs
print("Total time: {0} seconds".format(time.time() - start_time))

In [ ]:
# test the model
n_batches = int(mnist.test.num_examples/BATCH_SIZE)
total_correct_preds = 0

for i in range(n_batches):
    X_batch, Y_batch = mnist.test.next_batch(BATCH_SIZE)
    _, loss_batch, logits_batch = sess.run([train, loss, logits], 
                                    feed_dict={X: X_batch, Y:Y_batch, dropout: ???})
    
    preds = tf.nn.softmax(logits_batch)
    correct_preds = tf.equal(tf.argmax(preds, axis=1), tf.argmax(Y_batch, axis=1))
    accuracy = tf.reduce_sum(tf.cast(correct_preds, tf.float32))
    total_correct_preds += sess.run(accuracy)   

total_test_num = int(mnist.test.num_examples/BATCH_SIZE)*BATCH_SIZE
print("Accuracy {0}".format(total_correct_preds/total_test_num))